In [1]:

    
import cv2
import numpy as np
import os




#Change path according to where your data is
path = r"C:\Users\livin\Desktop\bbleap\smear"

def start_detection(dir_path):
    '''
    Identifies subdirectories from input folder and performs smear detection
    on the images in each subdirectory. Creates mean, intermediate and final images
	for each directory
	Input: path for data set
	'''
    print ('Starting...')
    directories = os.listdir(dir_path)
    dir_list = []
    for d in directories:
        if os.path.isdir(dir_path + "\\" + d):
            dir_list.append(d)
    print(dir_list)

    for directory in dir_list:
        
        if directory == 'middleSmear':
            temp_path = path + '\\' + directory
            data_set = process_image(temp_path)
            final_data_set = split_data(data_set)

            print ('Creating Mean Image for', directory)
            mean_img = create_mean_image(final_data_set)
            mean = cv2.imwrite(directory + '_mean_image.jpg',mean_img)
            #mean = cv2.imwrite(os.path.join(path, '_mean_image.jpg'),mean_img)

            print(mean)

            print ('Creating Intermediate Mask for', directory)
            prelim_mask = create_prelim_mask(mean_img)
            cv2.imwrite(directory + '_intermediate_mask.jpg',prelim_mask)
            print(prelim_mask)

            print ('Creating Mask for', directory)
            final_image = create_binary_mask(prelim_mask)
            cv2.imwrite(directory + '_final_mask.jpg',final_image)
        else:
            pass
    return final_image    

def process_image(dir_path):
    '''
	Pre-processes every image by converting to grayscale, applying histogram equalization,
	blurring and binary threshold. Returns list of processed images
	Input: directory path for images
	'''
    images = []
    for image in os.listdir(dir_path):
        img = cv2.imread(os.path.join(dir_path,image))
        if img is not None:
            gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            eq_img = cv2.equalizeHist(gray_image)
            blur_img = cv2.blur(eq_img, (3,3))
            ret,thresh_img = cv2.threshold(blur_img,127,255,cv2.THRESH_BINARY)
            images.append(thresh_img)
    return images
        

def split_data(image_set):
    '''
	Splits set of images into subsets of approximately 200 images for batch processing
	Input: list of images
	'''
    total = []
    count = 0
    number_of_splits = len(image_set)//20###
    #number_of_splits = 1
    for i in range(number_of_splits):
        maxcount = count + 20
        if maxcount < len(image_set)-1:
            temp_arr = image_set[count:maxcount]
        else:
            temp_arr = image_set[count:]
        total.append(temp_arr)
        count += 20
    return total


def calc_mean_image(arr, length):
    '''
	Returns an averaged image of the given list of images
	Input: list of images
	'''
    i = 1
    #print(arr)
        
    sum_image = arr[0] * 1/length
    while (i < len(arr)):
        sum_image = cv2.add(sum_image,arr[i]* 1/length)
        i += 1
    return sum_image


def create_mean_image(total_data):
    '''
	Batch processes the average of each image subset and returns one averaged image
	Input: List of lists of images
	'''
    avg_images = []
    for data in total_data:
        temp_img = calc_mean_image(data,len(data))
        avg_images.append(temp_img)
    #print(len(avg_images))###
    return calc_mean_image(avg_images,len(avg_images))
    

def create_prelim_mask(img):
    '''
	Applies an adaptive threshold on the input and performs a bitwise not operation
	Input: image
	'''
    img = img.astype(np.uint8)###
    adapt = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,105,10)
    return cv2.bitwise_not(adapt)
    

def create_binary_mask(img):
    '''
	Applies erosion and dilation to reduce noise and returns a final binary mask
	Input: image
	'''
    kernel = np.ones((10,10),np.uint8)
    erosion = cv2.erode(img,kernel,iterations = 5)
    dilation = cv2.dilate(erosion,kernel,iterations = 5)
    return dilation

#im = start_detection(path)

In [3]:
type(im)

numpy.ndarray

In [4]:
cv2.imshow("hi",im[0])

In [1]:
#### from linkedin, spacially varying blur detection
import blur_detector
import cv2
if __name__ == '__main__':
    img = cv2.imread(r"C:\Users\livin\Desktop\bbleap\smear\middleSmear\0ac94e44-b836-4cdc-ad76-f30d730f08fa.png", 0)
    blur_map = blur_detector.detectBlur(img, downsampling_factor=4, num_scales=4, scale_start=2, num_iterations_RF_filter=3)

    cv2.imwrite(r"C:\Users\livin\Desktop\bbleap\smear\linkedin.png",blur_map)
    #cv2.imshow('ori_img', img)
    cv2.imshow('blur_map', blur_map)
    
    
    

    cv2.waitKey(0)

[|                  ] 10%
[| |                ] 20%
[| | |              ] 30%
[| | | |            ] 40%
[| | | | |          ] 50%
[| | | | | |        ] 60%
[| | | | | | |      ] 70%
[| | | | | | | |    ] 80%
[| | | | | | | | |  ] 90%
[| | | | | | | | | |] 100%


C:\Users\livin\AppData\Local\Programs\Python\Python39\lib\site-packages\blur_detector\__init__.py:232: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  local_entropy = self.entropyFilt(T_max)


In [10]:
print(blur_map)

[[0.01763963 0.01763963 0.01769579 ... 0.07982506 0.07983355 0.07983355]
 [0.01763963 0.01763963 0.01769579 ... 0.07982506 0.07983355 0.07983355]
 [0.01764427 0.01764427 0.01770046 ... 0.08062963 0.08063171 0.08063171]
 ...
 [0.01138407 0.01138407 0.01138636 ... 0.02731062 0.0273099  0.0273099 ]
 [0.01138009 0.01138009 0.01138238 ... 0.02731012 0.0273094  0.0273094 ]
 [0.01138009 0.01138009 0.01138238 ... 0.02731012 0.0273094  0.0273094 ]]


In [13]:
rgb_transform = np.uint8(np.around(blur_map))
rgb_transform

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [4]:
import numpy as np

In [17]:
img = cv2.convertScaleAbs(img, alpha=(255.0))
cv2.imwrite(r"C:\Users\livin\Desktop\bbleap\smear\linkedin.png",img)

True

In [6]:
import cv2
import numpy as np
import blur_detector

In [7]:
m = blur_map/np.max(blur_map)
cv2.imshow('b', m)

In [7]:
img = cv2.imread(r"C:\Users\livin\Desktop\bbleap\smear\middleSmear\0ac94e44-b836-4cdc-ad76-f30d730f08fa.png", 0)
blur_map = blur_detector.detectBlur(img, downsampling_factor=4, num_scales=4, scale_start=2, num_iterations_RF_filter=3)

#from skimage import io
#io.imsave(r"C:\Users\livin\Desktop\bbleap\smear\linkedinio.png", blur_map)

[|                  ] 10%
[| |                ] 20%
[| | |              ] 30%
[| | | |            ] 40%
[| | | | |          ] 50%
[| | | | | |        ] 60%
[| | | | | | |      ] 70%
[| | | | | | | |    ] 80%
[| | | | | | | | |  ] 90%
[| | | | | | | | | |] 100%


In [4]:
##clean
img = cv2.imread(r"C:\Users\livin\Desktop\bbleap\clean\e7ed8bc7-ad21-4119-9f73-7016cb8dd3fe.png", 0)
blur_map = blur_detector.detectBlur(img, downsampling_factor=4, num_scales=4, scale_start=2, num_iterations_RF_filter=3)

from skimage import io
io.imsave(r"C:\Users\livin\Desktop\bbleap\smear\linkedinio_clean_img.png", blur_map)

[|                  ] 10%
[| |                ] 20%
[| | |              ] 30%
[| | | |            ] 40%
[| | | | |          ] 50%
[| | | | | |        ] 60%
[| | | | | | |      ] 70%
[| | | | | | | |    ] 80%
[| | | | | | | | |  ] 90%
[| | | | | | | | | |] 100%


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [9]:
p = create_prelim_mask(blur_map)
m = create_binary_mask(p)


In [13]:
cv2.imshow("name",m)
cv2.waitKey(0)

-1

In [16]:
io.imsave(r"C:\Users\livin\Desktop\bbleap\smear\linkedinio_mask.png", p)

C:\Users\livin\AppData\Local\Temp\ipykernel_88608\3784907212.py:1: UserWarning: C:\Users\livin\Desktop\bbleap\smear\linkedinio_mask.png is a low contrast image
  io.imsave(r"C:\Users\livin\Desktop\bbleap\smear\linkedinio_mask.png", p)


In [18]:
# import the modules
import os
from os import listdir
 
    
blurred_version = []
# get the path/directory
folder_dir = r"C:\Users\livin\Desktop\bbleap\smear\middleSmear"
for images in os.listdir(folder_dir):
 
    # check if the image ends with png
    if (images.endswith(".png")):
        img = cv2.imread(os.path.join(folder_dir,images), 0)
        blur_map = blur_detector.detectBlur(img, downsampling_factor=4, num_scales=4, scale_start=2, num_iterations_RF_filter=3)
        print(images)
        blurred_version.append(blur_map)

[|                  ] 10%
[| |                ] 20%
[| | |              ] 30%
[| | | |            ] 40%
[| | | | |          ] 50%
[| | | | | |        ] 60%
[| | | | | | |      ] 70%
[| | | | | | | |    ] 80%
[| | | | | | | | |  ] 90%
[| | | | | | | | | |] 100%
0a8ab14d-9f3c-4505-8e04-804eb7ad42e9.png
[|                  ] 10%
[| |                ] 20%
[| | |              ] 30%
[| | | |            ] 40%
[| | | | |          ] 50%
[| | | | | |        ] 60%
[| | | | | | |      ] 70%
[| | | | | | | |    ] 80%
[| | | | | | | | |  ] 90%
[| | | | | | | | | |] 100%
0aad2318-efaa-42ed-8cc8-7af52b7e00f5.png
[|                  ] 10%
[| |                ] 20%
[| | |              ] 30%
[| | | |            ] 40%
[| | | | |          ] 50%
[| | | | | |        ] 60%
[| | | | | | |      ] 70%
[| | | | | | | |    ] 80%
[| | | | | | | | |  ] 90%
[| | | | | | | | | |] 100%
0ab87707-58c2-40bc-9543-eb67ce852feb.png
[|                  ] 10%
[| |                ] 20%
[| | |              ] 30%
[| | | |        

[| | |              ] 30%
[| | | |            ] 40%
[| | | | |          ] 50%
[| | | | | |        ] 60%
[| | | | | | |      ] 70%
[| | | | | | | |    ] 80%
[| | | | | | | | |  ] 90%
[| | | | | | | | | |] 100%
0edab639-5380-415b-8929-b9145e267808.png


In [19]:
mi = calc_mean_image(blurred_version, len(blurred_version))
p = create_prelim_mask(mi)
m = create_binary_mask(p)

In [20]:
io.imsave(r"C:\Users\livin\Desktop\bbleap\smear\linkedinio_meanimage.png", mi)
io.imsave(r"C:\Users\livin\Desktop\bbleap\smear\linkedinio_prelimmask.png", p)
io.imsave(r"C:\Users\livin\Desktop\bbleap\smear\linkedinio_mask.png", m)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\livin\AppData\Local\Temp\ipykernel_88608\676162090.py:2: UserWarning: C:\Users\livin\Desktop\bbleap\smear\linkedinio_prelimmask.png is a low contrast image
  io.imsave(r"C:\Users\livin\Desktop\bbleap\smear\linkedinio_prelimmask.png", p)
C:\Users\livin\AppData\Local\Temp\ipykernel_88608\676162090.py:3: UserWarning: C:\Users\livin\Desktop\bbleap\smear\linkedinio_mask.png is a low contrast image
  io.imsave(r"C:\Users\livin\Desktop\bbleap\smear\linkedinio_mask.png", m)
